In [1]:
#!pip install ludwig
import pandas as pd
from ludwig.utils.data_utils import add_sequence_feature_column

In [2]:
df = pd.read_csv('../dataset/temperature.csv', usecols=['Los Angeles']).rename(columns={"Los Angeles": "temperature"}).fillna(
    method='backfill').fillna(method='ffill')

In [3]:
# normalize
df.temperature = ((df.temperature - df.temperature.mean()) / df.temperature.std())

In [4]:
train_size = int(0.6 * len(df))
vali_size = int(0.2 * len(df))

# train, validation, test split
df['split'] = 0
df.loc[((df.index.values >= train_size) & (df.index.values < train_size + vali_size)), ('split')] = 1
df.loc[df.index.values >= train_size + vali_size, ('split')] = 2

In [5]:
# prepare timeseries input feature colum
# (here we are using 20 preceeding values to predict the target)
add_sequence_feature_column(df, 'temperature', 20)

In [6]:
train = df.loc[df['split'] == 0, :]
valid = df.loc[df['split'] == 1, :]
test = df.loc[df['split'] == 2, :]

In [7]:
df.to_csv('../dataset/temperature_la.csv')
train.to_csv('../dataset/train.csv', header=True, index=False)
valid.to_csv('../dataset/valid.csv', header=True, index=False)
test.to_csv('../dataset/test.csv', header=True, index=False)